# Clean Indicators 

One of the most important things we will be using in this projects are the **indicators**. They will be used to represent the main objectives of the **text files** (*papers* or *official documents*) we are going to be analysing, plus we want to *match* the **general indicators** with the **cultural ones** so we can see how far they are from each other. This is the main reason for which we want to have the indicators **as clean as possible**, as it will be our way of measuring if the model results are behaving well enough, and so we will be preparing them as our first step.

Some indicators contain **non-alphanumeric** characters, **bad encoding** symbols and way **too specific words** (such as dates). In order to improve the results of the model we will **remove** / **replace** all these characters and errors. There are also some **non-english indicators** that we are not targeting in this project so we will **translate** them and then remove the possible **duplicates** we find after all the changes we made.

**Cleaning Summary**:

* Remove wrong characters 

* Translate non-english indicators

* Drop duplicates from both types

## Environment

### Libraries

In [1]:
# Base libraries
import pandas as pd

# Visualization
from IPython.display import Markdown, display

# Text and Language
import regex as re
from langdetect import detect
from textblob import TextBlob

### Data Extraction

In [2]:
path = 'Indicators/'
filename = path + 'indicators.xlsx'

general_indicators = pd.read_excel(filename, 
                                   sheet_name='Indic Generales', 
                                   engine='openpyxl')["indicator_en"].rename('indicator')

cultural_indicators = pd.read_excel(filename, 
                                    sheet_name='Indic Culturales', 
                                    engine='openpyxl')["indicator_en"].rename('indicator')

## Cleaning

### Error characters

As we portrayed before, there are a few errors in the indicators due to their encoding and format. The main ones we will be adressing are the following:

*  **\xa0** (byte characters not properly recognized due to the encoding)
*  **(±SD)** (The sum - minus symbol might not be recognized, neither the standard deviation and even if they were they're very generic)
*  **2006** (years, dates are not useful when  working with topics and general indicators)
*  **(REI)** (Remove empty parenthesis or parenthesis with only one word that are often used as a summarization of the main sentence)

Further cleaning includes removing some unnecesary characters such as **squared brackets** or extra **unnecesary spaces**. 

In order to do that we will use **RegEx**, a popular tool that allows us to modify (including *replacing* and *removing*) characters given some custom defined circumstances and standard encoding Python methods to work with latin encoding (**ISO-8859-1**).

In [3]:
def clean_indicator(text):
    
    # Encode in latin if this char (error) is not found
    if ('–' not in text) & ('€' not in text):
        try:
            text = text.encode('latin', 'ignore').decode('UTF-8')
        except: 
            pass
    
    # Remove years
    text = re.sub(r'[0-9]{4}', " ", text.strip())    
    
    # Replace the following rules with spaces
    replace_rules = [
        r"\xa0", # Error for smaller numbers 
        r" *± *SD", # Standard Deviaton (not useful)
        r"\*",# * Characters
        r" +", # Remove extra spaces
        r"[0-9]\\[0-9a-zA-Z]+",
    ]
    
    for rule in replace_rules:
        text = re.sub(rule, " ", text)
                
    # Remove the following rules
    remove_rules = [
        r'(\[|\()[a-zA-Z0-9%]*(\]|\))', # Remove parenthesis containing only one word
        r'\( *\)', # Remove empty Parenthesis
    ]
    
    for rule in remove_rules:
        text = re.sub(rule, "", text)
    
    # Fix unmatched parenthesis
    if text.count('(') == text.count(')'):
        return text.strip()
    else:
        return text.strip() +')'


clean_general_indicators = general_indicators.fillna("").apply(clean_indicator)
clean_cultural_indicators = cultural_indicators.fillna("").apply(clean_indicator)

#### Checking

Now that we have cleaned our indicators we can check a **sample of 10** of them that changed after the processing so we can tell there is indeed a difference. The **left** column will be the **clean** one, while the **right** column will be the **raw** one: 

##### 10 Sample modified General Indicators

In [4]:
display(Markdown(pd.concat([clean_general_indicators.rename('clean_indicator'), 
           general_indicators], 
          axis=1)[clean_general_indicators != general_indicators].sample(10, random_state=42).to_markdown()))

|     | clean_indicator                                                                                                                                                                   | indicator                                                                                                                                                                               |
|----:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 104 | Residential energy use                                                                                                                                                            | Residential energy use (11)                                                                                                                                                             |
|  15 | Public facilities and spaces                                                                                                                                                      | Public facilities and spaces                                                                                                                                                            |
| 570 | Safe and healthy sport sites                                                                                                                                                      | Safe and healthy sport sites                                                                                                                                                            |
| 795 | Contractors Health, safety and enivironmental pre-qualification                                                                                                                   | Contractors Health, safety and enivironmental pre-qualification                                                                                                                         |
| 849 | Google hits for the string city name & climate change & urban heat island (hits per million inhabitants)                                                                          | Google hits for the string “city name & climate change & urban heat island” (hits per million inhabitants)                                                                              |
| 785 | Health, safety and enivironmental management systems                                                                                                                              | Health, safety and enivironmental management systems                                                                                                                                    |
| 776 | Public access to Health, safety and enivironmental information                                                                                                                    | Public access to Health, safety and enivironmental information                                                                                                                          |
| 559 | Number of complaints about hygienic conditions                                                                                                                                    | Number of complaints about hygienic conditions (SI)                                                                                                                                     |
|  31 | Percentage of park management staff who possess qualifications related to ecology, environmental management, landscape architecture, or other park management-related disciplines | Percentage of park management staff who possess qualifications related to ecology, environmental management, landscape architecture, or other park management-related disciplines (MII) |
| 406 | Capacity of public transport                                                                                                                                                      | Capacity of public transport                                                                                                                                                            |

We can see that there are some improvements in removing unnecesary characters overall so we will **keep the new changes**. This doesn't mean that we will have no errors whatsoever, there might have been removed information but it's the price we have to pay as unstructured data can't always be perfectly generalized and there will be always be some errors.

##### 10 Sample modified Cultural Indicators

In [5]:
display(Markdown(pd.concat([clean_cultural_indicators.rename('clean_indicator'), 
           cultural_indicators],
          axis=1)[clean_cultural_indicators != cultural_indicators].sample(10, random_state=42).to_markdown()))

|     | clean_indicator                                                                              | indicator                                                                                     |
|----:|:---------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------|
|  61 | Commercial activities linked to Crats, Artisans, Craftsmanship,                              | Commercial activities linked to Crats, Artisans, Craftsmanship,                               |
|  65 | Cultural events related to olive farming                                                     | Cultural events related to olive farming (%)                                                  |
| 115 | Promotion and education in Past Identities programmes and spaces                             | Promotion and education in  Past Identities’ programmes and spaces                            |
| 211 | Respect for residents life-style                                                             | Respect for residents’ life-style                                                             |
| 193 | New identities programmes and spaces. Alternative forms of celebrating contemporary culture. | New identities’ programmes and spaces. Alternative forms of celebrating contemporary culture. |
| 210 | Respect for residents traditional culture                                                    | Respect for residents’ traditional culture                                                    |
| 130 | Institutional management of International fests or expert meetings                           | Institutional management of  International fests or expert meetings                           |
| 206 | Type and amount of training given to tourism employees                                       | Type and amount of training given to tourism employees (guide)                                |
|  78 | Creative Uses for Old Buildings                                                              | Creative Uses for Old Buildings (Entrepreneurs)                                               |
| 135 | Number of citizens initiatives                                                               | Number of citizen’s initiatives                                                               |

We can appreciate that **cultural indicators** change less than general ones, mostly because they contain more typos and unnecesary characters instead of years or way too much specific information, although we keep removing some unnecesary words inside the parenthesis.

### Language Translation

For translating the indicators we will be using the library *langdetect*, but it's way too far from being perfect and for this reason we will need to check just the **latin languages** (as germanic languages often confuse the module) as most of the non-english indicators are **spanish** ones, but can be often confused with **portuguese**, **italian**, **french** or **romanian**. 

Once the indicators are tagged they will be translated to english if one of their tags is a latin one and the result will be compared to the original indicator and if it doesn't match it will be reviewed manually.

In [6]:
def textblob_translate(sentence, from_lang):
    
    if from_lang in ["es", "it", "pt", "fr", "ro"]:
        try:
            return TextBlob(sentence).translate(to='en', from_lang=from_lang)
        except:
            return sentence 
    else: 
        return sentence

#### General Indicators

In [7]:
clean_general_indicators = clean_general_indicators[clean_general_indicators != ''].reset_index(drop=True)
lang_general_indicators = pd.concat([clean_general_indicators, clean_general_indicators.apply(detect).rename('language')], axis=1)
translated = lang_general_indicators.apply(lambda x: textblob_translate(x["indicator"], x["language"]), axis=1).apply(str)
final_general_indicators = pd.concat([translated.rename("translated_indicator"), lang_general_indicators], axis=1)

##### 10 Sample translated  indicators

In [8]:
display(Markdown(final_general_indicators[final_general_indicators["translated_indicator"] !=
                         final_general_indicators["indicator"]].sample(10, random_state=42).to_markdown()))

|     | translated_indicator                             | indicator                                        | language   |
|----:|:-------------------------------------------------|:-------------------------------------------------|:-----------|
| 535 | Financial Administration                         | Financial administration                         | pt         |
| 223 | Sustainable Resource Management                  | Sustainable resource management                  | fr         |
|  61 | Sustainable Natural Resource Management          | Sustainable natural resource management          | fr         |
| 133 | Development Outside Cities                       | Development outside cities                       | fr         |
| 393 | Average Travel Timea                             | Average travel timea                             | it         |
| 504 | Expenditure on Infrastructure                    | Expenditure on infrastructurea                   | ro         |
| 705 | Public Participation                             | Public participation                             | it         |
|  96 | Solid residue / efficiency                       | Residuo sólido /Eficiencia                       | es         |
| 718 | Participation in Cultural and Leisure Activities | Participation in cultural and leisure activities | ro         |
| 179 | Road Surface Per Capita (M 2)                    | Road surface per capita (m 2 )                   | ro         |

With this sample we can see that some indicators get a little bit weird but it's not a huge problem as we are **translating the spanish ones properly**.

#### Cultural Indicators

In [9]:
clean_cultural_indicators = clean_cultural_indicators[clean_cultural_indicators != ''].reset_index(drop=True)
lang_cultural_indicators = pd.concat([clean_cultural_indicators, clean_cultural_indicators.apply(detect).rename('language')], axis=1)
translated = lang_cultural_indicators.apply(lambda x: textblob_translate(x["indicator"], x["language"]), axis=1).apply(str)
final_cultural_indicators = pd.concat([translated.rename("translated_indicator"), lang_cultural_indicators], axis=1)

##### 10 Sample translated  indicators

In [10]:
display(Markdown(final_cultural_indicators[final_cultural_indicators["translated_indicator"] !=
                          final_cultural_indicators["indicator"]].sample(10, random_state=42).to_markdown()))

|     | translated_indicator                                    | indicator                                                 | language   |
|----:|:--------------------------------------------------------|:----------------------------------------------------------|:-----------|
| 122 | Cultural Cultural Exchange                              | Increase cultural exchange                                | ro         |
| 160 | Historic Monument Values                                | Historic monument values                                  | ro         |
|   3 | Inspirational Landscapes                                | Inspirational landscapes                                  | it         |
| 222 | Memorial Stones                                         | Memorial stones                                           | it         |
| 100 | EFFECT ON AREA REVITALIZATION / DEVELOPMENT             | Effect on area revitalization/development                 | ro         |
| 118 | Cultural, Leisure and Sport Facilities, Programs, Races | Cultural, leisure and sport facilities, programs, courses | fr         |
|   9 | Spiritual Sites                                         | Spiritual sites                                           | it         |
|   7 | Beautiful, Attractive Place                             | Beautiful, attractive place                               | fr         |
| 256 | L / T Ratio                                             | L/T ratio                                                 | fr         |
|  96 | Cultural Sites Maintenance Level                        | Cultural sites maintenance level                          | fr         |

In this case it's not a big deal and we might be distorting some indicators so we will just revert the changes to the **raw indicators**.

### Removing Duplicates &  Exporting

We have previously removed duplicates but now we will be focusing in the duplicates **between cultural and general indicators**. If any of them is shared by both, it will be removed from the **general indicators** (as it is more specific to understand that it is actually a cultural one).

Now the indicators are clean enough to start working with them, we will export them in a **two columns** dataframe containing the translated, **clean indicators** and their relative **indicator type** (general-cultural). The next step will be the model selection and the analysis of the relationship between the indicators.

In [11]:
export_general_indicators = pd.concat([final_general_indicators["translated_indicator"].rename("indicator"), 
                           pd.Series(['general' for row in range(len(final_general_indicators))]).rename("type")], 
                           axis=1)

export_cultural_indicators = pd.concat([final_cultural_indicators["indicator"], 
                           pd.Series(['cultural' for row in range(len(final_cultural_indicators))]).rename("type")], 
                           axis=1)

duplicate_values = pd.concat([export_general_indicators, 
                              export_cultural_indicators]).reset_index(drop=True)["indicator"].str.lower().drop_duplicates()

pd.concat([export_general_indicators, 
           export_cultural_indicators]).reset_index(drop=True).loc[duplicate_values.index].to_csv(path + "clean_indicators.csv", index=False)

In [12]:
#!jupyter nbconvert "Clean Indicators.ipynb" --to html_toc --TemplateExporter.exclude_input=True -TagRemovePreprocessor.remove_cell_tags="hide" 